<font size="3"> I show how to create surnames and first names datasets from the data of the Facebook leak of 2021. It covers about 100 countries. Not putting the link here but it can be found easily in torrent format.

# Extracting the data

The data takes the form of multiple compressed files, one for each country. The compression format is either zip, rar or 7zip. The files are in csv format with either a comma or colon separator. Sometimes the below function didn't work because the file was too big or invalid or the format was different. In this case I took a custom approach not detailed here. 

In [1]:
from glob import glob
import io
import re
import zipfile
import rarfile
import os 
from string import punctuation, ascii_letters
import numpy as np
import pandas as pd

def get_sep(file): #The separator is sometimes a comma and sometimes a colon. This function finds the kind of separator of the file.
    head_a = str(file[:5000])
    comma_count = head_a.count(",")
    colon_count = head_a.count(":")
    if comma_count > colon_count :
        sep = ","
    else:
        sep=":"
    return sep


def df_from_archive(archive):
    df_list = []
    files = archive.namelist()  #the data is usually split into multiple files inside the archive
    
    for file in files : 
        print(file)
        
        with archive.open(file) as myfile:
            a = myfile.read()
            sep = get_sep(a)

            try:
                chunks_list = [] #loading files in chunks to avoid crashing               
                for chunk in pd.read_table(io.StringIO(a.decode("utf-8")),sep=sep,header=None,usecols=[2,3,4], quoting=3,engine="c",dtype=pd.StringDtype(),chunksize=1000000):
                    chunks_list.append(chunk)
                df = pd.concat(chunks_list)
                
            except:
                chunks_list = []
                print("failure,switching to python engine")  #sometimes the c engine will crash, in that case the python engine can sometimes do the trick
                for chunk in pd.read_table(io.StringIO(a.decode("utf-8")),sep=sep,header=None,usecols=[2,3,4], quoting=3,engine="python",dtype=pd.StringDtype(),chunksize=1000000):
                    chunks_list.append(chunk)
                df = pd.concat(chunks_list)

            df.columns = ["first_name","surname","gender"]
            df_list.append(df)
            
    df = pd.concat(df_list)
    return df


def get_df(compressed_file):
    try:
        if ".zip" in compressed_file:
            with zipfile.ZipFile(compressed_file, 'r') as archive:
                df = df_from_archive(archive)

        elif ".rar" in compressed_file:
            with rarfile.RarFile(compressed_file, 'r') as archive: 
                df = df_from_archive(archive)
    except:
        print("File error : ", compressed_file)  
        
    return df

I create two dataframes for each country, one for first names, one for surnames.

In [ ]:
files = glob("facebook/*.rar") + glob("facebook/*.zip")
print(len(files), " countries")

for directory in ["surnames", "first_names"]:
    if not os.path.exists(directory):
        os.makedirs(directory)

for n,file in enumerate(files) :
    country = file.replace("facebook/",'').replace(".zip","").replace(".rar","").lower()
    print('n°' + str(n+1) +" : " + country)
    
    df = get_df(file)
    
    surnames = df["surname"].value_counts().rename("count")
    print("len surnames : ", len(surnames))
    surnames.to_csv(f'surnames/{country}_surnames.csv')
    
    #also extracting a first name dataset just in case
    first_names = df[["first_name","gender"]].value_counts().rename("count")
    print("len first names : ", len(first_names))
    first_names.to_csv(f'first_names/{country}_first_names.csv')
               
    print(country, " done",end="\n\n")
